In [36]:
import uproot
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

# "browser" gives better viewing than "jupyterlab", but its up to you
# import plotly.io as pio
# pio.renderers.default = "browser"

#Parse file
file = uproot.open("ntuple_653.root")
event = file["rechitntupler/hits/event"].array(library="np")
xdata = file["rechitntupler/hits/rechit_x"].array(library="np")
ydata = file["rechitntupler/hits/rechit_y"].array(library="np")
zdata = file["rechitntupler/hits/rechit_z"].array(library="np")
layer = file["rechitntupler/hits/rechit_layer"].array(library="np")
energy = file["rechitntupler/hits/rechit_energy"].array(library="np")
chip = file["rechitntupler/hits/rechit_chip"].array(library="np")
channel = file["rechitntupler/hits/rechit_channel"].array(library="np")
trueBeamEnergy = file["rechitntupler/hits/trueBeamEnergy"].array(library="np")

#Total num of events in first 28 layers
totcount = 0
for j in layer[0]:
    if j<29:
        totcount +=1
    else:
        break

        
#Display intensity defined by highest energy
normalizN = max(energy[0][:totcount])

xdatahigh = []
xdatalow = []
ydatahigh = []
ydatalow = []
zdatahigh = []
zdatalow = []
energyhigh = []
energylow = []
#separate lists to greatly save on figure load/render time
for num in range(totcount):
    erg = energy[0][num]/normalizN
    if erg < 0.02:
        xdatalow.append(xdata[0][num])
        ydatalow.append(ydata[0][num])
        zdatalow.append(zdata[0][num])
        energylow.append(energy[0][num])
    else:
        xdatahigh.append(xdata[0][num])
        ydatahigh.append(ydata[0][num])
        zdatahigh.append(zdata[0][num])
        energyhigh.append(energy[0][num])
    
#initialize the Figure (3d scatter, colorbar)
fig = make_subplots(rows = 1, 
                    cols = 2, 
                    specs = [[{'type' : 'scene'},{'type' : 'xy'}]], 
                    column_widths = [0.95,0.05])
    
#Add the lower energy vals on a single trace (opacity approximation)
fig.add_trace(go.Scatter3d(x=xdatalow[:len(xdatalow)],
                            y = ydatalow[:len(ydatalow)],
                            z = zdatalow[:len(zdatalow)],
                            mode='markers',
                            marker=dict(color = 'blue', 
                                        opacity = 0.1,
                                        symbol ='circle',
                                        size = 10)), row = 1, col = 1)
    
#high E vals as separate traces for opacity/color control. 
for num in range(len(xdatahigh)):
    erg = energyhigh[num]/normalizN
    if erg < 0.5:
        r = round(255*(erg*2))
        b = round((1-2*erg) * 255)
        g = 0
    else:
        r= 255
        b = 0
        g = round((erg-0.5)*2 *255)
    opp = erg*0.9 + 0.1
    fig.add_trace(go.Scatter3d(x=np.asarray(xdatahigh[num]),
                               y = np.asarray(ydatahigh[num]),
                               z = np.asarray(zdatahigh[num]),
                               mode='markers',
                               marker=dict(color = ['rgba({}, {}, {}, {})'.format(r,g,b,opp)],
                                           symbol ='circle',
                                           size = 10)), row = 1, col = 1)

    
#Make the color scale
image = []
for num in range(250):
    row = []
    perc = (num)/250
    for i in range(25):
        pixel = []
        if perc < 0.5:
            pixel.append(round(255*(perc*2)))
            pixel.append(0)
            pixel.append(round((1-2*perc) * 255))
        else:
            pixel.append(255)
            pixel.append(round((perc-0.5)*2 *255))
            pixel.append(0)
        row.append(pixel)
    image.append(row)
imager = np.array(image, dtype=np.uint8)

fig.add_trace(go.Image(z=imager),row = 1, col = 2)
fig.update_layout(yaxis1=dict(tickvals = [250, 200, 150, 100, 50, 0], 
                              title_text = 'Energy'), 
                  xaxis1=dict(showticklabels=False))
    
#set the camera orientation
multip = 0.8
fig.update_scenes(aspectmode = "manual",
                  aspectratio=dict(x=1,y=1,z=5), 
                  camera=dict(eye=dict(x=2*multip, y=-1*multip, z=-5*multip),
                              up=dict(x=0, y=0, z=0),
                              center=dict(x=-1,y=0.5,z=0)), row = 1, col = 1)

fig.show()

#better yet is saving it as your own file
# pio.write_image(fig, /path/to/file.png, width = 1920, height = 1080)
